## Part 1: Preprocessing

In [1]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model
from tensorflow.keras import layers
import os
os.environ["ROCM_PATH"]="/opt/rocm"

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

2025-03-17 09:36:09.308053: E external/local_xla/xla/stream_executor/plugin_registry.cc:91] Invalid plugin kind specified: FFT
2025-03-17 09:36:09.338587: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: SSE3 SSE4.1 SSE4.2 AVX AVX2 AVX512F AVX512_VNNI AVX512_BF16 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2025-03-17 09:36:09.607635: E external/local_xla/xla/stream_executor/plugin_registry.cc:91] Invalid plugin kind specified: DNN


,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [2]:
# Determine the number of unique values in each column
print(attrition_df.columns)
attrition_df.nunique()


Index(['Age', 'Attrition', 'BusinessTravel', 'Department', 'DistanceFromHome',
       'Education', 'EducationField', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager'],
      dtype='object')


Age                         43
Attrition                    2
BusinessTravel               3
Department                   3
DistanceFromHome            29
Education                    5
EducationField               6
EnvironmentSatisfaction      4
HourlyRate                  71
JobInvolvement               4
JobLevel                     5
JobRole                      9
JobSatisfaction              4
MaritalStatus                3
NumCompaniesWorked          10
OverTime                     2
PercentSalaryHike           15
PerformanceRating            2
RelationshipSatisfaction     4
StockOptionLevel             4
TotalWorkingYears           40
TrainingTimesLastYear        7
WorkLifeBalance              4
YearsAtCompany              37
YearsInCurrentRole          19
YearsSinceLastPromotion     16
YearsWithCurrManager        18
dtype: int64

In [3]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[["Attrition", "Department"]]

In [4]:
# Create a list of at least 10 column names to use as X data
x_dat_list = ['Age', 'BusinessTravel', 'DistanceFromHome',
       'Education', 'EducationField', 'EnvironmentSatisfaction', 'HourlyRate',
       'JobInvolvement', 'JobLevel', 'JobRole', 'JobSatisfaction',
       'MaritalStatus', 'NumCompaniesWorked', 'OverTime', 'PercentSalaryHike',
       'PerformanceRating', 'RelationshipSatisfaction', 'StockOptionLevel',
       'TotalWorkingYears', 'TrainingTimesLastYear', 'WorkLifeBalance',
       'YearsAtCompany', 'YearsInCurrentRole', 'YearsSinceLastPromotion',
       'YearsWithCurrManager']

# Create X_df using your selected columns
X_df = attrition_df[x_dat_list]

# Show the data types for X_df
print(X_df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1470 entries, 0 to 1469
Data columns (total 25 columns):
 #   Column                    Non-Null Count  Dtype 
---  ------                    --------------  ----- 
 0   Age                       1470 non-null   int64 
 1   BusinessTravel            1470 non-null   object
 2   DistanceFromHome          1470 non-null   int64 
 3   Education                 1470 non-null   int64 
 4   EducationField            1470 non-null   object
 5   EnvironmentSatisfaction   1470 non-null   int64 
 6   HourlyRate                1470 non-null   int64 
 7   JobInvolvement            1470 non-null   int64 
 8   JobLevel                  1470 non-null   int64 
 9   JobRole                   1470 non-null   object
 10  JobSatisfaction           1470 non-null   int64 
 11  MaritalStatus             1470 non-null   object
 12  NumCompaniesWorked        1470 non-null   int64 
 13  OverTime                  1470 non-null   object
 14  PercentSalaryHike       

In [5]:
# Split the data into training and testing sets
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X_df, y_df)

In [6]:
# Convert your X data to numeric data types however you see fit
# The function below will first detect columns which are string values and then fit a one hot encoder to said column
hot_encoders = []
drop_cols = []
for i in x_train.columns:
    curr_sel = x_train[i]
    if type(curr_sel[1]) == str:
        curr_sel = np.expand_dims(curr_sel, axis=1)
        new_enc = OneHotEncoder()
        new_enc.fit(curr_sel)
        drop_cols.append(i)
        hot_encoders.append(new_enc)

# Then the following function will be used to apply the fitted encoder to the columns
# The corresponding columns are then dropped and their transformed columns are appended back to the original data as a numpy array
def drop_n_encode(x_val, hot_encoders, drop_cols):
    trans_encoded = []
    track = 0
    for i in x_val.columns:
        if i in drop_cols:
            curr_sel = x_val[i]
            curr_sel = np.expand_dims(curr_sel, axis=1)
            encode = hot_encoders[track]
            track = track + 1
            transformed = encode.transform(curr_sel).toarray()
            trans_encoded.append(transformed)
            
    hot_x_val = x_val.drop(columns=drop_cols)
    hot_x_val = hot_x_val.to_numpy()
    for t in trans_encoded:
        hot_x_val = np.append(hot_x_val, t, axis=1)
    return hot_x_val
print(x_train.shape)
hot_x_train = drop_n_encode(x_train, hot_encoders, drop_cols)
print(hot_x_train.shape)
print(hot_x_train[0, :])
hot_x_test = drop_n_encode(x_test, hot_encoders, drop_cols)

(1102, 25)
(1102, 43)
[35.  2.  4.  4. 61.  3.  2.  1.  2. 11.  3.  1.  1. 16.  2.  4.  1.  0.
  0.  0.  1.  0.  0.  0.  1.  0.  0.  0.  0.  1.  0.  0.  0.  0.  0.  0.
  0.  0.  1.  0.  0.  1.  0.]


In [7]:
# Create a StandardScaler
scaley = StandardScaler()

# Fit the StandardScaler to the training data
scaley.fit(hot_x_train)

# Scale the training and testing data
x_train_scaled = scaley.transform(hot_x_train)
x_test_scaled = scaley.transform(hot_x_test)

In [8]:
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder for the Department column
dep_enc = OneHotEncoder()

# Fit the encoder to the training data
dep_enc.fit(np.expand_dims(y_train['Department'], axis=1))

# Create two new variables by applying the encoder
# to the training and testing data
dep_y_train_hot = dep_enc.transform(np.expand_dims(y_train['Department'], axis=1)).toarray()
dep_y_test_hot = dep_enc.transform(np.expand_dims(y_test['Department'], axis=1)).toarray()

In [9]:
# Create a OneHotEncoder for the Attrition column
atr_enc = OneHotEncoder()

# Fit the encoder to the training data
atr_enc.fit(np.expand_dims(y_train['Attrition'], axis=1))

# Create two new variables by applying the encoder
# to the training and testing data
atr_y_train_hot = atr_enc.transform(np.expand_dims(y_train['Attrition'], axis=1)).toarray()
atr_y_test_hot = atr_enc.transform(np.expand_dims(y_test['Attrition'], axis=1)).toarray()


## Part 2: Create, Compile, and Train the Model

In [25]:
# Find the number of columns in the X training data.
x_col_num = x_train_scaled.shape[1]

# Create the input layer
ins = layers.Input(shape=(x_col_num, ))

# Create at least two shared layers
shared_lay1 = layers.Dense(units=128, activation = "relu")(ins)
shared_lay2 = layers.Dense(units = 128, activation = "relu")(shared_lay1)

In [26]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
dep_hid = layers.Dense(units=32, activation = "relu")(shared_lay2)

# Create the output layer
dep_out = layers.Dense(units = dep_y_test_hot.shape[1], activation="softmax", name="deparment_out")(dep_hid)

In [27]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
atr_hid = layers.Dense(units=32, activation = "relu")(shared_lay2)

# Create the output layer
atr_out = layers.Dense(units = atr_y_test_hot.shape[1], activation="softmax", name="attrition_out")(atr_hid)


In [28]:
# Create the model
mod = Model(inputs=ins, outputs =[dep_out, atr_out])

# Compile the model
mod.compile(optimizer = "adam", loss = "binary_crossentropy", metrics=['accuracy', 'accuracy'])

# Summarize the model
mod.summary()

Model: "functional_2"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_2       │ (None, 43)        │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_12 (Dense)    │ (None, 128)       │      5,632 │ input_layer_2[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_13 (Dense)    │ (None, 128)       │     16,512 │ dense_12[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_14 (Dense)    │ (None, 32)        │      4,128 │ dense_13[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_15 (Dense)    │ (None, 32)        │      4,128 │ dense_13[0][0]    │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ deparment_out       │ (None, 3)         │         99 │ dense_14[0][0]    │
│ (Dense)             │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ attrition_out       │ (None, 2)         │         66 │ dense_15[0][0]    │
│ (Dense)             │                   │            │                   │
└─────────────────────┴───────────────────┴────────────┴───────────────────┘

 Total params: 30,565 (119.39 KB)

 Trainable params: 30,565 (119.39 KB)

 Non-trainable params: 0 (0.00 B)

In [29]:
# Train the model
mod.fit(x_train_scaled, [dep_y_train_hot, atr_y_train_hot], epochs = 100)

Epoch 1/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 2s 16ms/step - attrition_out_accuracy: 0.8343 - attrition_out_loss: 0.5217 - deparment_out_accuracy: 0.5971 - deparment_out_loss: 0.5795 - loss: 1.1016
Epoch 2/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_out_accuracy: 0.8317 - attrition_out_loss: 0.3871 - deparment_out_accuracy: 0.9264 - deparment_out_loss: 0.1958 - loss: 0.5828
Epoch 3/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_out_accuracy: 0.8550 - attrition_out_loss: 0.3345 - deparment_out_accuracy: 0.9575 - deparment_out_loss: 0.0777 - loss: 0.4124
Epoch 4/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_out_accuracy: 0.8729 - attrition_out_loss: 0.3075 - deparment_out_accuracy: 0.9823 - deparment_out_loss: 0.0438 - loss: 0.3515
Epoch 5/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/step - attrition_out_accuracy: 0.8808 - attrition_out_loss: 0.2688 - deparment_out_accuracy: 0.9900 - deparment_out_loss: 0.0286 - loss: 0.2975
Epoch 6/100
35/35 ━━━━━━━━━━━━━━━━━━━━ 0s 3ms/ste

In [30]:
# Evaluate the model with the testing data
evaluations = mod.evaluate(x_test_scaled, [dep_y_test_hot, atr_y_test_hot])

12/12 ━━━━━━━━━━━━━━━━━━━━ 1s 25ms/step - attrition_out_accuracy: 0.9003 - attrition_out_loss: 0.9401 - deparment_out_accuracy: 0.9764 - deparment_out_loss: 0.1150 - loss: 1.0286   


In [ ]:
# Print the accuracy for both department and attrition
accuracy_tr, accuracy_dep = evaluations[-2:]
print(f"The resultant attrition accuracy is: {accuracy_tr} \n The resultant department accuracy is: {accuracy_dep}")

[1.2834293842315674, 0.10843276977539062, 1.3471870422363281, 0.8722826242446899, 0.97826087474823]


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. 
2. 
3. 